In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth

In [2]:
from oauth2client.client import GoogleCredentials

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
#merged datasets 1
#https://drive.google.com/file/d/1ngefCHpfiIeRI7boNrTqv7Qvb8x18ee4/view?usp=sharing

fileDownloaded = drive.CreateFile({"id":"1ngefCHpfiIeRI7boNrTqv7Qvb8x18ee4"})

In [6]:
fileDownloaded.GetContentFile("merge.csv")

In [7]:
import pandas as pd
data = pd.read_csv('merge.csv', encoding='utf-8')

In [8]:
data.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,ZIP,TSP: Type of Match (Enhanced),Number of Children (Enhanced),Family composition (Enhanced),Gender code - Person 1,Advantage Individual Marital Status - Person 1,Advantage Individual Marital Status Indicator - Person 1,Person Sequence Number - Person 1,Age in 2-year ranges - Person 1 (Enhanced),Target Home Market Value 2.0 Code,Target Home Market Value 2.0 Indicator,NICHES 5.0 Code,NICHES 5.0 Indicator,Advantage Dwelling Type,Advantage Dwelling Type Indicator,Advantage Home Owner,Advantage Home Owner Indicator,Advantage Length of Residence,Advantage Length of Residence Indicator,Advantage Household Age (Enhanced),Advantage Household Age Indicator (Enhanced),Advantage Presence of Children (Enhanced),Advantage Presence of Children Indicator (Enhanced),Advantage Target Income 3.0,Advantage Target Income 3.0 Indicator,Discretionary Spending Income,Target Net Worth 4.0 Code,Target Net Worth 4.0 Indicator,Household Key,psn,",given_name",middle_initial,surname,contracted_address,city,state,zip,...,sc_warehouse_clubs_13-15m_trans,sc_warehouse_clubs_13-15m_spend,sc_warehouse_clubs_16-18m_trans,sc_warehouse_clubs_16-18m_spend,sc_warehouse_clubs_19-21m_trans,sc_warehouse_clubs_19-21m_spend,sc_warehouse_clubs_22-24m_trans,sc_warehouse_clubs_22-24m_spend,sc_wine_and_liquor_0-3m_trans,sc_wine_and_liquor_0-3m_spend,sc_wine_and_liquor_4-6m_trans,sc_wine_and_liquor_4-6m_spend,sc_wine_and_liquor_7-9m_trans,sc_wine_and_liquor_7-9m_spend,sc_wine_and_liquor_10-12m_trans,sc_wine_and_liquor_10-12m_spend,sc_wine_and_liquor_13-15m_trans,sc_wine_and_liquor_13-15m_spend,sc_wine_and_liquor_16-18m_trans,sc_wine_and_liquor_16-18m_spend,sc_wine_and_liquor_19-21m_trans,sc_wine_and_liquor_19-21m_spend,sc_wine_and_liquor_22-24m_trans,sc_wine_and_liquor_22-24m_spend,sc_wireless_0-3m_trans,sc_wireless_0-3m_spend,sc_wireless_4-6m_trans,sc_wireless_4-6m_spend,sc_wireless_7-9m_trans,sc_wireless_7-9m_spend,sc_wireless_10-12m_trans,sc_wireless_10-12m_spend,sc_wireless_13-15m_trans,sc_wireless_13-15m_spend,sc_wireless_16-18m_trans,sc_wireless_16-18m_spend,sc_wireless_19-21m_trans,sc_wireless_19-21m_spend,sc_wireless_22-24m_trans,sc_wireless_22-24m_spend
0,0,164,5358626,22314,A,NaN,6.0,1.0,2.0,H,3300572A0311940000A51A,62.0,890.0,S,N1,S,5.0,S,4.0,S,8.0,S,5.0,S,0.0,H,7,H,4,7,H,3.900337e+09,3300572A0311940000A51A,"3231631,RANDALL",C,BERG,400 MADISON ST APT 801,ALEXANDRIA,VA,22314,...,NaN,NaN,NaN,NaN,2.0,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,F,NaN,NaN,NaN,NaN,1.0,8,1.0,9,1.0,8,1.0,3,NaN,NaN
1,1,251,5358713,22314,A,NaN,12.0,1.0,2.0,H,3300572A0311FA019775B8,60.0,1476.0,S,I1,S,1.0,S,4.0,S,5.0,S,5.0,S,0.0,H,C,H,A,A,H,5.089726e+08,3300572A0311FA019775B8,"3254921,JOHN",Q,BINFORD,125 MONTGOMERY PL,ALEXANDRIA,VA,22314,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,8,2.0,5,2.0,1,2.0,2,1.0,1,2.0,2,3.0,8,3.0,A,1.0,8,1.0,3,1.0,I,1.0,1,1.0,1,1.0,1,1.0,D,NaN,NaN
2,2,278,5358740,22314,A,NaN,2.0,2.0,1.0,S,330055F3030E0C000B3592,72.0,1240.0,S,C3,S,5.0,S,4.0,S,4.0,S,6.0,S,0.0,H,D,S,9,B,H,2.183194e+09,330055F3030E0C000B3592,"3252778,LESLIE",L,BAER,824 WATER PL,ALEXANDRIA,VA,22314,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,282,5358744,22314,A,NaN,2.0,2.0,1.0,S,3300572A0310CF0001DD4F,40.0,1313.0,S,D1,S,5.0,S,4.0,S,5.0,S,4.0,S,1.0,H,D,H,C,9,H,4.782153e+07,3300572A0310CF0001DD4F,"3324540,MICHELLE",M,MCDUFFIE,822 WATER PL,ALEXANDRIA,VA,22314,...,2.0,6,NaN,NaN,2.0,A,NaN,NaN,4.0,5,3.0,2,3.0,7,3.0,D,2.0,6,3.0,H,2.0,7,3.0,5,1.0,7,NaN,NaN,NaN,NaN,1.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,374,5358836,22314,A,NaN,12.0,1.0,2.0,S,3300571D0310C7000083BD,42.0,479.0,A,J3,S,5.0,S,NaN,NaN,5.0,S,3.0,S,0.0,H,6,S,4,3,H,3.742277e+09,3300571D0310C7000083BD,"3231203,ERIC",J,LARRONDO,801 N PITT ST APT 1007,ALEXANDRIA,VA,22314,...,3.0,9,NaN,NaN,1.0,2,NaN,NaN,1.0,4,2.0,3,1.0,9,3.0,3,1.0,1,1.0,1,2.0,1,1.0,1,3.0,F,1.0,4,2.0,

generate some variables based on consumptions on categories

clean the merged data

In [9]:
import seaborn as sns
import pandas as pd
import numpy as np
import os

In [10]:
#delete indicators that do not contain relevant information
data.filter(regex = 'Indicator$', axis= 1)


,Target Home Market Value 2.0 Indicator,NICHES 5.0 Indicator,Advantage Dwelling Type Indicator,Advantage Home Owner Indicator,Advantage Length of Residence Indicator,Advantage Target Income 3.0 Indicator,Target Net Worth 4.0 Indicator
0,S,S,S,S,S,H,H
1,S,S,S,S,S,H,H
2,S,S,S,S,S,S,H
3,S,S,S,S,S,H,H
4,A,S,S,NaN,S,S,H
...,...,...,...,...,...,...,...
150931,H,S,S,S,S,H,H
150932,H,S,S,S,S,H,H
150933,S,S,S,S,S,H,H
150934,S,S,S,S,S,H,H


In [11]:
data.drop(columns = data.filter(regex = 'Indicator$', axis= 1), inplace= True)

In [12]:
data.head()


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,ZIP,TSP: Type of Match (Enhanced),Number of Children (Enhanced),Family composition (Enhanced),Gender code - Person 1,Advantage Individual Marital Status - Person 1,Advantage Individual Marital Status Indicator - Person 1,Person Sequence Number - Person 1,Age in 2-year ranges - Person 1 (Enhanced),Target Home Market Value 2.0 Code,NICHES 5.0 Code,Advantage Dwelling Type,Advantage Home Owner,Advantage Length of Residence,Advantage Household Age (Enhanced),Advantage Household Age Indicator (Enhanced),Advantage Presence of Children (Enhanced),Advantage Presence of Children Indicator (Enhanced),Advantage Target Income 3.0,Discretionary Spending Income,Target Net Worth 4.0 Code,Household Key,psn,",given_name",middle_initial,surname,contracted_address,city,state,zip,zip_plus4,household_id,file_code,overall_0-3m_trans,overall_0-3m_spend,overall_4-6m_trans,overall_4-6m_spend,...,sc_warehouse_clubs_13-15m_trans,sc_warehouse_clubs_13-15m_spend,sc_warehouse_clubs_16-18m_trans,sc_warehouse_clubs_16-18m_spend,sc_warehouse_clubs_19-21m_trans,sc_warehouse_clubs_19-21m_spend,sc_warehouse_clubs_22-24m_trans,sc_warehouse_clubs_22-24m_spend,sc_wine_and_liquor_0-3m_trans,sc_wine_and_liquor_0-3m_spend,sc_wine_and_liquor_4-6m_trans,sc_wine_and_liquor_4-6m_spend,sc_wine_and_liquor_7-9m_trans,sc_wine_and_liquor_7-9m_spend,sc_wine_and_liquor_10-12m_trans,sc_wine_and_liquor_10-12m_spend,sc_wine_and_liquor_13-15m_trans,sc_wine_and_liquor_13-15m_spend,sc_wine_and_liquor_16-18m_trans,sc_wine_and_liquor_16-18m_spend,sc_wine_and_liquor_19-21m_trans,sc_wine_and_liquor_19-21m_spend,sc_wine_and_liquor_22-24m_trans,sc_wine_and_liquor_22-24m_spend,sc_wireless_0-3m_trans,sc_wireless_0-3m_spend,sc_wireless_4-6m_trans,sc_wireless_4-6m_spend,sc_wireless_7-9m_trans,sc_wireless_7-9m_spend,sc_wireless_10-12m_trans,sc_wireless_10-12m_spend,sc_wireless_13-15m_trans,sc_wireless_13-15m_spend,sc_wireless_16-18m_trans,sc_wireless_16-18m_spend,sc_wireless_19-21m_trans,sc_wireless_19-21m_spend,sc_wireless_22-24m_trans,sc_wireless_22-24m_spend
0,0,164,5358626,22314,A,NaN,6.0,1.0,2.0,H,3300572A0311940000A51A,62.0,890.0,N1,5.0,4.0,8.0,5.0,S,0.0,H,7,4,7,3.900337e+09,3300572A0311940000A51A,"3231631,RANDALL",C,BERG,400 MADISON ST APT 801,ALEXANDRIA,VA,22314,1747,5.122314e+16,P,7.0,Z,7.0,O,...,NaN,NaN,NaN,NaN,2.0,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,F,NaN,NaN,NaN,NaN,1.0,8,1.0,9,1.0,8,1.0,3,NaN,NaN
1,1,251,5358713,22314,A,NaN,12.0,1.0,2.0,H,3300572A0311FA019775B8,60.0,1476.0,I1,1.0,4.0,5.0,5.0,S,0.0,H,C,A,A,5.089726e+08,3300572A0311FA019775B8,"3254921,JOHN",Q,BINFORD,125 MONTGOMERY PL,ALEXANDRIA,VA,22314,1768,5.122314e+16,P,7.0,Z,7.0,Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,8,2.0,5,2.0,1,2.0,2,1.0,1,2.0,2,3.0,8,3.0,A,1.0,8,1.0,3,1.0,I,1.0,1,1.0,1,1.0,1,1.0,D,NaN,NaN
2,2,278,5358740,22314,A,NaN,2.0,2.0,1.0,S,330055F3030E0C000B3592,72.0,1240.0,C3,5.0,4.0,4.0,6.0,S,0.0,H,D,9,B,2.183194e+09,330055F3030E0C000B3592,"3252778,LESLIE",L,BAER,824 WATER PL,ALEXANDRIA,VA,22314,1771,5.122314e+16,P,5.0,O,6.0,O,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,282,5358744,22314,A,NaN,2.0,2.0,1.0,S,3300572A0310CF0001DD4F,40.0,1313.0,D1,5.0,4.0,5.0,4.0,S,1.0,H,D,C,9,4.782153e+07,3300572A0310CF0001DD4F,"3324540,MICHELLE",M,MCDUFFIE,822 WATER PL,ALEXANDRIA,VA,22314,1771,5.122314e+16,P,7.0,Z,7.0,Z,...,2.0,6,NaN,NaN,2.0,A,NaN,NaN,4.0,5,3.0,2,3.0,7,3.0,D,2.0,6,3.0,H,2.0,7,3.0,5,1.0,7,NaN,NaN,NaN,NaN,1.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,374,5358836,22314,A,NaN,12.0,1.0,2.0,S,3300571D0310C7000083BD,42.0,479.0,J3,5.0,NaN,5.0,3.0,S,0.0,H,6,4,3,3.742277e+09,3300571D0310C7000083BD,"3231203,ERIC",J,LARRONDO,801 N PITT ST APT 1007,ALEXANDRIA,VA,22314,1792,5.122314e+16,P,7.0,Z,7.0,Z,...,3.0,9,NaN,NaN,1.0,2,NaN,NaN,1.0,4,2.0,3,1.0,9,3.0,3,1.0,1,1.0,1,2.0,1,1.0,1,3.0,F,1.0,4,2.0,8,3.0,F,1.0,5,3.0,9,3.0,G,2.0,2


In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [14]:
x= data[['Number of Children (Enhanced)','Advantage Target Income 3.0','Discretionary Spending Income','Target Net Worth 4.0 Code']]

In [15]:
x['Target Net Worth 4.0 Code'] = x['Target Net Worth 4.0 Code'].replace(['A','B'],[10,11])

In [16]:
x['Advantage Target Income 3.0'] = x['Advantage Target Income 3.0'].replace(['A','B', 'C', 'D'],[10,11, 12, 13])

In [17]:
x['Discretionary Spending Income'] = x['Discretionary Spending Income'].replace(['A','B', 'C'],[10,11, 12])

In [18]:
from sklearn.preprocessing import MinMaxScaler

In [19]:
num_features = ['Number of Children (Enhanced)','Advantage Target Income 3.0','Discretionary Spending Income','Target Net Worth 4.0 Code']
num_transformer = Pipeline(
    steps=[("imputer",IterativeImputer(initial_strategy='median',imputation_order='ascending')),
           ('scaler',MinMaxScaler())]
)

preprocessor = ColumnTransformer(
    transformers=[("num",num_transformer,num_features)]
)

In [20]:
ouch=pd.DataFrame(preprocessor.fit_transform(x),
                  columns = ['Number of Children (Enhanced)','Advantage Target Income 3.0',
                             'Discretionary Spending Income','Target Net Worth 4.0 Code'])

In [21]:
ouch.isnull().sum()

Number of Children (Enhanced)    0
Advantage Target Income 3.0      0
Discretionary Spending Income    0
Target Net Worth 4.0 Code        0
dtype: int64

In [22]:
ouch

,Number of Children (Enhanced),Advantage Target Income 3.0,Discretionary Spending Income,Target Net Worth 4.0 Code
0,0.183862,0.500000,0.333333,0.636364
1,0.192241,0.916667,0.833333,0.909091
2,0.295834,1.000000,0.750000,1.000000
3,0.164031,1.000000,1.000000,0.818182
4,0.142291,0.416667,0.333333,0.272727
...,...,...,...,...
150931,0.248611,1.000000,0.833333,1.000000
150932,0.085201,0.416667,0.416667,0.454545
150933,0.109807,1.000000,1.000000,0.909091
150934,0.168964,1.000000,1.000000,0.727273


In [23]:
x2= data[['Family composition (Enhanced)', 'Advantage Individual Marital Status - Person 1', 'Gender code - Person 1','Advantage Dwelling Type','Advantage Home Owner','Advantage Length of Residence']]

In [24]:
x2.isnull().sum().sort_values(ascending=False)

Advantage Home Owner                              10446
Advantage Dwelling Type                            7427
Advantage Individual Marital Status - Person 1     6310
Gender code - Person 1                             5236
Family composition (Enhanced)                      5140
Advantage Length of Residence                      4985
dtype: int64

In [25]:
num_features2 = ['Family composition (Enhanced)', 'Advantage Individual Marital Status - Person 1', 
                'Gender code - Person 1','Advantage Dwelling Type','Advantage Home Owner','Advantage Length of Residence']
num_transformer2 = Pipeline(
    steps=[("imputer",IterativeImputer(initial_strategy='median',imputation_order='ascending'))]
)

preprocessor2 = ColumnTransformer(
    transformers=[("num",num_transformer2,num_features2)]
)

In [26]:
ouch2=pd.DataFrame(preprocessor2.fit_transform(x2),
                  columns = ['Family composition (Enhanced)', 'Advantage Individual Marital Status - Person 1', 
                             'Gender code - Person 1','Advantage Dwelling Type','Advantage Home Owner',
                             'Advantage Length of Residence'])

In [27]:
ouch2.isnull().sum()

Family composition (Enhanced)                     0
Advantage Individual Marital Status - Person 1    0
Gender code - Person 1                            0
Advantage Dwelling Type                           0
Advantage Home Owner                              0
Advantage Length of Residence                     0
dtype: int64

In [28]:
ouch2

,Family composition (Enhanced),Advantage Individual Marital Status - Person 1,Gender code - Person 1,Advantage Dwelling Type,Advantage Home Owner,Advantage Length of Residence
0,6.0,2.0,1.0,5.0,4.000000,8.0
1,12.0,2.0,1.0,1.0,4.000000,5.0
2,2.0,1.0,2.0,5.0,4.000000,4.0
3,2.0,1.0,2.0,5.0,4.000000,5.0
4,12.0,2.0,1.0,5.0,3.555655,5.0
...,...,...,...,...,...,...
150931,14.0,1.0,2.0,5.0,4.000000,7.0
150932,14.0,2.0,2.0,5.0,4.000000,2.0
150933,1.0,1.0,1.0,1.0,4.000000,7.0
150934,2.0,1.0,2.0,1.0,4.000000,5.0


### Recode spend codes' indices into average values

In [29]:
spend = data.filter(regex = 'spend',axis= 1)

In [30]:
spend1 = spend.replace(['1','2','3','4','5','6','7','8','9','A','B','C','D',
                        'E','F','G','H','I','J','K','L','M','N','O','P','Q','R',
                        'S','T','U','V','W','X','Y','Z']
                       ,[25.50,62.50,88.00,112.50,137.50,162.50,187.50,225.00,
                         275.00,325.00,375.00,430.00,475.00,525.00,575.00,625.00,
                         675.00,725.00,775.00,825.00,875.00,925.00,975.00,1125.00,
                         1375.00,1625.00,1875.00,2125.00,2375.00,2625.00,2875.00,
                         3125.00,3375.00,3625.00,3750.00])

In [31]:
spend1

,overall_0-3m_spend,overall_4-6m_spend,overall_7-9m_spend,overall_10-12m_spend,overall_13-15m_spend,overall_16-18m_spend,overall_19-21m_spend,overall_22-24m_spend,c_automotive_0-3m_spend,c_automotive_4-6m_spend,c_automotive_7-9m_spend,c_automotive_10-12m_spend,c_automotive_13-15m_spend,c_automotive_16-18m_spend,c_automotive_19-21m_spend,c_automotive_22-24m_spend,c_communications_service_providers_0-3m_spend,c_communications_service_providers_4-6m_spend,c_communications_service_providers_7-9m_spend,c_communications_service_providers_10-12m_spend,c_communications_service_providers_13-15m_spend,c_communications_service_providers_16-18m_spend,c_communications_service_providers_19-21m_spend,c_communications_service_providers_22-24m_spend,c_education_0-3m_spend,c_education_4-6m_spend,c_education_7-9m_spend,c_education_10-12m_spend,c_education_13-15m_spend,c_education_16-18m_spend,c_education_19-21m_spend,c_education_22-24m_spend,c_entertainment_0-3m_spend,c_entertainment_4-6m_spend,c_entertainment_7-9m_spend,c_entertainment_10-12m_spend,c_entertainment_13-15m_spend,c_entertainment_16-18m_spend,c_entertainment_19-21m_spend,c_entertainment_22-24m_spend,...,sc_travel_services_0-3m_spend,sc_travel_services_4-6m_spend,sc_travel_services_7-9m_spend,sc_travel_services_10-12m_spend,sc_travel_services_13-15m_spend,sc_travel_services_16-18m_spend,sc_travel_services_19-21m_spend,sc_travel_services_22-24m_spend,sc_video-pc_game_stores_0-3m_spend,sc_video-pc_game_stores_4-6m_spend,sc_video-pc_game_stores_7-9m_spend,sc_video-pc_game_stores_10-12m_spend,sc_video-pc_game_stores_13-15m_spend,sc_video-pc_game_stores_16-18m_spend,sc_video-pc_game_stores_19-21m_spend,sc_video-pc_game_stores_22-24m_spend,sc_warehouse_clubs_0-3m_spend,sc_warehouse_clubs_4-6m_spend,sc_warehouse_clubs_7-9m_spend,sc_warehouse_clubs_10-12m_spend,sc_warehouse_clubs_13-15m_spend,sc_warehouse_clubs_16-18m_spend,sc_warehouse_clubs_19-21m_spend,sc_warehouse_clubs_22-24m_spend,sc_wine_and_liquor_0-3m_spend,sc_wine_and_liquor_4-6m_spend,sc_wine_and_liquor_7-9m_spend,sc_wine_and_liquor_10-12m_spend,sc_wine_and_liquor_13-15m_spend,sc_wine_and_liquor_16-18m_spend,sc_wine_and_liquor_19-21m_spend,sc_wine_and_liquor_22-24m_spend,sc_wireless_0-3m_spend,sc_wireless_4-6m_spend,sc_wireless_7-9m_spend,sc_wireless_10-12m_spend,sc_wireless_13-15m_spend,sc_wireless_16-18m_spend,sc_wireless_19-21m_spend,sc_wireless_22-24m_spend
0,3750.0,1125.0,2125.0,3750.0,975.0,1875.0,3375.0,2125.0,88.0,475.0,112.5,88.0,88.0,25.5,62.5,25.5,975.0,62.5,275.0,625.0,275.0,225.0,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3125.0,62.5,25.5,325.0,62.5,62.5,1375.0,162.5,...,275.0,NaN,25.5,NaN,NaN,NaN,112.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,225.0,25.5,88.0,25.5,NaN,NaN,475.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,575.0,NaN,NaN,225.0,275.0,225.0,88.0,NaN
1,3750.0,3750.0,3375.0,3750.0,3750.0,3750.0,3750.0,3750.0,112.5,NaN,430.0,NaN,225.0,NaN,88.0,NaN,430.0,187.5,775.0,575.0,137.5,225.0,575.0,225.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.5,62.5,25.5,62.5,25.5,275.0,187.5,112.5,...,NaN,875.0,NaN,325.0,NaN,NaN,375.0,475.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,225.0,137.5,25.5,62.5,25.5,62.5,225.0,325.0,225.0,88.0,725.0,25.5,25.5,25.5,475.0,NaN
2,1125.0,1125.0,375.0,1125.0,3750.0,775.0,1375.0,825.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3750.0,3750.0,3750.0,3750.0,3750.0,3750.0,3750.0,3750.0,325.0,325.0,725.0,225.0,137.5,375.0,825.0,225.0,325.0,275.0,112.5,275.0,162.5,137.5,162.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,225.0,25.5,187.5,25.5,88.0,25.5,25.5,25.5,...,1375.0,NaN,25.5,NaN,430.0,112.5,625.0,275.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,975.0,112.5,275.0,NaN,162.5,NaN,325.0,NaN,137.5,62.5,187.5,475.0,162.5,675.0,187.5,137.5,187.5,NaN,NaN,25.5,NaN,NaN,NaN,NaN
4,

### Filling in missing values in *spend*

In [35]:
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

In [42]:
big=pd.DataFrame(SimpleImputer(strategy='mean').fit_transform(spend1),columns=spend1.columns)

In [43]:
big.isnull().sum()

overall_0-3m_spend          0
overall_4-6m_spend          0
overall_7-9m_spend          0
overall_10-12m_spend        0
overall_13-15m_spend        0
                           ..
sc_wireless_10-12m_spend    0
sc_wireless_13-15m_spend    0
sc_wireless_16-18m_spend    0
sc_wireless_19-21m_spend    0
sc_wireless_22-24m_spend    0
Length: 640, dtype: int64

### Keep the highest categories of *spend*

In [64]:
automotive = big.filter(regex = 'c_automotive',axis= 1)
communication = big.filter(regex = 'c_communication',axis= 1)
education = big.filter(regex = 'c_education',axis= 1)
entertainment = big.filter(regex = 'c_entertainment',axis= 1)
financial_services = big.filter(regex = 'c_financial_services',axis= 1)
food_convenience_drug_store = big.filter(regex = 'c_food-convenience-drug_store',axis= 1)
other_retail = big.filter(regex = 'c_other_retail',axis= 1)
restaurant = big.filter(regex = "c_restaurant", axis=1)
retail = big.filter(regex = "c_retail", axis=1)
services = big.filter(regex = "c_services", axis=1)
travel = big.filter(regex = "c_travel", axis=1)

In [65]:
spend_clean=pd.concat([automotive, communication, education, entertainment,
                       financial_services, food_convenience_drug_store,
                       other_retail, restaurant, retail, services, travel], axis=1)

In [71]:
spend_clean.drop(columns = spend_clean.filter(regex = 'sc_education_', axis= 1), inplace= True)
spend_clean.drop(columns = spend_clean.filter(regex = 'sc_travel_services_', axis= 1), inplace= True)

In [72]:
for col in spend_clean.columns:
  print(col)

c_automotive_0-3m_spend
c_automotive_4-6m_spend
c_automotive_7-9m_spend
c_automotive_10-12m_spend
c_automotive_13-15m_spend
c_automotive_16-18m_spend
c_automotive_19-21m_spend
c_automotive_22-24m_spend
c_communications_service_providers_0-3m_spend
c_communications_service_providers_4-6m_spend
c_communications_service_providers_7-9m_spend
c_communications_service_providers_10-12m_spend
c_communications_service_providers_13-15m_spend
c_communications_service_providers_16-18m_spend
c_communications_service_providers_19-21m_spend
c_communications_service_providers_22-24m_spend
c_education_0-3m_spend
c_education_4-6m_spend
c_education_7-9m_spend
c_education_10-12m_spend
c_education_13-15m_spend
c_education_16-18m_spend
c_education_19-21m_spend
c_education_22-24m_spend
c_entertainment_0-3m_spend
c_entertainment_4-6m_spend
c_entertainment_7-9m_spend
c_entertainment_10-12m_spend
c_entertainment_13-15m_spend
c_entertainment_16-18m_spend
c_entertainment_19-21m_spend
c_entertainment_22-24m_spend


In [73]:
spend_clean.head()

,c_automotive_0-3m_spend,c_automotive_4-6m_spend,c_automotive_7-9m_spend,c_automotive_10-12m_spend,c_automotive_13-15m_spend,c_automotive_16-18m_spend,c_automotive_19-21m_spend,c_automotive_22-24m_spend,c_communications_service_providers_0-3m_spend,c_communications_service_providers_4-6m_spend,c_communications_service_providers_7-9m_spend,c_communications_service_providers_10-12m_spend,c_communications_service_providers_13-15m_spend,c_communications_service_providers_16-18m_spend,c_communications_service_providers_19-21m_spend,c_communications_service_providers_22-24m_spend,c_education_0-3m_spend,c_education_4-6m_spend,c_education_7-9m_spend,c_education_10-12m_spend,c_education_13-15m_spend,c_education_16-18m_spend,c_education_19-21m_spend,c_education_22-24m_spend,c_entertainment_0-3m_spend,c_entertainment_4-6m_spend,c_entertainment_7-9m_spend,c_entertainment_10-12m_spend,c_entertainment_13-15m_spend,c_entertainment_16-18m_spend,c_entertainment_19-21m_spend,c_entertainment_22-24m_spend,c_financial_services_0-3m_spend,c_financial_services_4-6m_spend,c_financial_services_7-9m_spend,c_financial_services_10-12m_spend,c_financial_services_13-15m_spend,c_financial_services_16-18m_spend,c_financial_services_19-21m_spend,c_financial_services_22-24m_spend,...,c_other_retail_0-3m_spend,c_other_retail_4-6m_spend,c_other_retail_7-9m_spend,c_other_retail_10-12m_spend,c_other_retail_13-15m_spend,c_other_retail_16-18m_spend,c_other_retail_19-21m_spend,c_other_retail_22-24m_spend,c_restaurant_0-3m_spend,c_restaurant_4-6m_spend,c_restaurant_7-9m_spend,c_restaurant_10-12m_spend,c_restaurant_13-15m_spend,c_restaurant_16-18m_spend,c_restaurant_19-21m_spend,c_restaurant_22-24m_spend,c_retail_0-3m_spend,c_retail_4-6m_spend,c_retail_7-9m_spend,c_retail_10-12m_spend,c_retail_13-15m_spend,c_retail_16-18m_spend,c_retail_19-21m_spend,c_retail_22-24m_spend,c_services_0-3m_spend,c_services_4-6m_spend,c_services_7-9m_spend,c_services_10-12m_spend,c_services_13-15m_spend,c_services_16-18m_spend,c_services_19-21m_spend,c_services_22-24m_spend,c_travel_0-3m_spend,c_travel_4-6m_spend,c_travel_7-9m_spend,c_travel_10-12m_spend,c_travel_13-15m_spend,c_travel_16-18m_spend,c_travel_19-21m_spend,c_travel_22-24m_spend
0,88.000000,475.000000,112.50000,88.000000,88.000000,25.500000,62.500000,25.500000,975.000000,62.500000,275.000000,625.00000,275.000000,225.000000,88.000000,377.654667,2.051387,3.071523,1.939815,2.791896,8.789127,16.04058,22.212941,16.203936,3125.000000,62.500000,25.500000,325.000000,62.500,62.500000,1375.000000,162.500000,566.58608,573.412982,560.278492,564.307895,538.501651,529.8868,537.999523,526.431143,...,62.5,225.000000,475.0,325.000000,25.500000,876.836671,325.0,850.967388,88.0,62.5,137.5,88.000000,62.5,88.0,112.5,25.5,825.0,137.5,575.0,1875.0,225.0,1125.0,875.0,430.0,191.518002,25.500000,207.234766,192.297749,188.388957,197.574786,203.188487,198.215601,275.000000,949.008245,25.500000,913.72279,812.636423,1030.355687,275.000000,25.500000
1,112.500000,369.427564,430.00000,378.178918,225.000000,361.464201,88.000000,362.471363,430.000000,187.500000,775.000000,575.00000,137.500000,225.000000,575.000000,225.000000,2.051387,3.071523,1.939815,2.791896,8.789127,16.04058,22.212941,16.203936,25.500000,62.500000,25.500000,62.500000,25.500,275.000000,187.500000,112.500000,566.58608,573.412982,560.278492,564.307895,538.501651,529.8868,537.999523,526.431143,...,475.0,925.000000,525.0,525.000000,430.000000,775.000000,475.0,1125.000000,875.0,775.0,430.0,375.000000,575.0,275.0,575.0,675.0,1375.0,575.0,875.0,2125.0,1125.0,1125.0,2125.0,225.0,191.518002,183.582818,207.234766,192.297749,137.500000,197.574786,25.500000,198.215601,375.000000,925.000000,225.000000,475.00000,25.500000,137.500000,775.000000,875.000000
2,393.355771,369.427564,381.74834,378.178918,358.914894,361.464201,382.961622,362.471363,427.652229,427.901942,436.662106,434.54812,409.645372,399.784611,414.279971,377.654667,2.051387,3.071523,1.939815,2.791896,8.789127,16.04058,22.212941,16.203936,259.166